In [2]:
import numpy as np
import scipy.signal as signal
import matplotlib.pyplot as plt
%matplotlib qt5

import time

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from loadbackprojection import *

In [4]:
libraryFilename = "ccpp/libbackprojection/liblibbackprojection.so"

In [5]:
lib = LibBackProjection( libraryFilename )

# Backprojection

## Load the analytic signal

In [ ]:
data_dir = "/home/pleroy/DATA/SIMU"
analyticSignal = np.load( data_dir + "/analyticSignal_0_19.npy" )
print( "analyticSignal.shape = {}".format(analyticSignal.shape) )

## Range focalization

In [ ]:
srf = np.fft.ifft( analyticSignal, axis=1 )
print( "srf.shape = {}".format(srf.shape) )

## Image geometry

In [ ]:
x_min = 200
x_max = 500
r_min = 80
r_max = 120

#extent=[horizontal_min,horizontal_max,vertical_min,vertical_max]
extent=[r_min,r_max,x_max,x_min]

d_x = 1.
d_r = 1.

x = np.arange( x_min, x_max, d_x )
x = x.reshape( x.size )
r = np.arange( r_min, r_max, d_r )
r = r.reshape( r.size )

## Azimuth focalization

In [ ]:
B = 150e6
c = 3e8
nbFiles = 20
# vehicule is plane
T = 600e-6
rampsPerFile = 1500
V = 40

In [ ]:
overSamplingRatio = 10
Nf = srf.shape[1]
Nover = overSamplingRatio * Nf
rangeResolution = c / (2 * B)
r_base = np.arange( Nf ) * rangeResolution
r_over = np.arange( Nover ) * rangeResolution / overSamplingRatio
print( "range from {:.2f}m to {:.2f}m, resolution = {}m, oversampled = {}m, ".format(
    r_over[0], r_over[-1], rangeResolution, rangeResolution / overSamplingRatio ) )

In [ ]:
Naz = nbFiles * rampsPerFile
xa_vec = np.arange( Naz ) * T * V

print( "x.shape = {}, r.shape = {}".format( x.shape, r.shape ) )
print( "x from {} to {}, r from {} to {}".format(x[0], x[-1], r[0], r[-1]) )

phi = 6 * np.pi / 180

nbLoops = xa_vec.shape[0]
progress = int( nbLoops / 10 )

### backProjectionOmp

In [ ]:
t = time.time()

img1  = np.zeros( (x.size, r.size), dtype=complex )
print( "img.shape = {}".format( img1.shape ) )

endK = Naz
dr = r_over[1] - r_over[0]
lib.so.backProjectionOmp( x, x.size,
              r, r.size,
              r_over, r_over.size, dr,
              srf.reshape(srf.size), endK, Nf, # endK instead of Naz
              xa_vec, img1.reshape(img1.size) )

elapsed = time.time() - t
print("execution time = " + str(elapsed))

### backProjection2

In [ ]:
t = time.time()

img2  = np.zeros( (x.size, r.size), dtype=complex )
print( "img.shape = {}".format( img2.shape ) )

endK = Naz
dr = r_over[1] - r_over[0]
lib.so.backProjection2( x, x.size,
              r, r.size,
              r_over, r_over.size, dr,
              srf.reshape(srf.size), Naz, Nf, # endK or Naz
              xa_vec, img2.reshape(img2.size) )

elapsed = time.time() - t
print("execution time = " + str(elapsed))

### backProjectionOmpGroundRange

In [ ]:
lib.reload()

In [ ]:
firstFile = 0
nbFiles = 20
lastFile = firstFile + nbFiles - 1
# build positions for all ramps
hPlane = 90
xyz = np.zeros( ( nbFiles * rampsPerFile, 5 ) )
xyz[ :, 2 ] = np.arange( 0, nbFiles * rampsPerFile ) * T * V
xyz[ :, 3 ] = 0
xyz[ :, 4 ] = hPlane

In [ ]:
t = time.time()

imgGroundRange  = np.zeros( (x.size, r.size), dtype=complex )
print( "img.shape = {}".format( img2.shape ) )

endK = Naz
dr = r_over[1] - r_over[0]
lib.so.backProjectionOmpGroundRange( x, x.size,
              r, r.size,
              r_over, r_over.size, dr,
              srf.reshape(srf.size), Naz, Nf, # endK or Naz
              xyz.reshape(xyz.size), imgGroundRange.reshape(imgGroundRange.size),
                                   hPlane)

elapsed = time.time() - t
print("execution time = " + str(elapsed))

In [ ]:
#(2.2391811659142823e-05+1.1432034209496042e-05j)
#(2.2391811659142823e-05+1.1432034209496042e-05j)
img1[0,0], imgGroundRange[0,0]

In [ ]:
plt.figure()
plt.imshow( 20 * np.log10( np.abs( imgGroundRange ) ), cmap='jet', extent=extent )
plt.grid()
plt.colorbar(orientation="horizontal")

# Interpolation

In [ ]:
Nx = 3000

xp = np.linspace(-10*np.pi, 10*np.pi, Nx)
dx = xp[1] - xp[0]
fp = Nx * np.sin(xp) + 1j * np.arange(Nx)

In [ ]:
x = np.linspace(-np.pi/10, np.pi/10, 100)
y = np.zeros( x.shape, dtype=complex )
idx = 0
for val in x:
    aux = mydll.interp( val, xp, fp, dx )
    y[idx] = aux.real + aux.imag
    idx += 1

In [ ]:
x = 0.5
y = mydll.interp( x, xp, fp, dx )

In [ ]:
dx

In [ ]:
y.real, y.imag

In [ ]:
plt.figure()
plt.subplot(211)
plt.plot( xp, np.real(fp), '.-' )
plt.plot( x, y.real, 'Dy' )
plt.subplot(212)
plt.plot( xp, np.imag(fp), '.-' )
plt.plot( x, y.imag, 'Dy' )

# Resampling

## With ctypes

In [6]:
Nx = 3000
upSampling = 10
Ny = Nx * upSampling

teta = np.linspace(-10*np.pi, 10*np.pi, Nx)
x = Nx * np.sin(teta) + 1j * np.arange(Nx)
fftx = np.zeros( Nx, dtype=complex )
y = np.zeros( Ny, dtype=complex )
ffty = np.zeros( y.shape, dtype=complex )

In [7]:
import timeit
import time

In [8]:
#%timeit signal.resample( x, Ny )

In [9]:
lib.reload()

In [10]:
#%timeit mydll.resample( x, fftx, Nx, y, ffty, Ny )
lib.so.resample( x, fftx, Nx, y, ffty, Ny )

1500

In [11]:
tx = np.arange(Nx)
ty = np.arange(Ny)/upSampling

plt.figure()

plt.plot(tx, np.real(x), 'o-', label="x real")
plt.plot(tx, np.imag(x), 'o-', label="x imag")
plt.plot(tx, np.abs(x), 'o-',label="x abs")

plt.plot(ty, np.real(y), '.-', label="y real oversampled")
plt.plot(ty, np.imag(y), '.-', label="y imag oversampled")
plt.plot(ty, np.abs(y), '.-', label="y abs oversampled")

plt.legend()

In [12]:
plt.figure()
plt.plot( np.real(x) - np.real(y[::upSampling]) )
plt.plot( np.imag(x) - np.imag(y[::upSampling]) )

## Other resampling

In [18]:
lib.reload()

In [19]:
# compare resample4 (exact) with resample4b (vec_ind is not correct)
fftx = np.fft.fft( x )
lib.so.resample4b( fftx, Nx, y, ffty, Ny )

1501

In [20]:
tx = np.arange(Nx)
ty = np.arange(Ny)/upSampling

plt.figure()

plt.plot(tx, np.real(x), 'o-', label="x real")
plt.plot(tx, np.imag(x), 'o-', label="x imag")
plt.plot(tx, np.abs(x), 'o-',label="x abs")

plt.plot(ty, np.real(y), '.-', label="y real oversampled")
plt.plot(ty, np.imag(y), '.-', label="y imag oversampled")
plt.plot(ty, np.abs(y), '.-', label="y abs oversampled")

plt.legend()

In [21]:
plt.figure()
plt.plot( np.real(x) - np.real(y[::upSampling]) )
plt.plot( np.imag(x) - np.imag(y[::upSampling]) )

## With numpy

In [ ]:
nbPoints = 1000
teta = np.linspace(-10*np.pi, 10*np.pi, nbPoints)
a = nbPoints * np.sin(teta) + 1j * np.arange(nbPoints)

In [ ]:
plt.figure()
plt.plot(np.real(a), label="real")
plt.plot(np.imag(a), label="imag")
plt.plot(np.abs(a), label="abs")
plt.legend()

In [ ]:
fft_a = np.fft.fft(a)
upSampling = 4

In [ ]:
fft_b = np.zeros(fft_a.shape[0]*upSampling, dtype=complex)
nbPoints2 = int( nbPoints / 2 ) + nbPoints%2
fft_b[0:nbPoints2] = fft_a[0:nbPoints2]
fft_b[-nbPoints2:] = fft_a[-nbPoints2:]
b = np.fft.ifft( fft_b ) * 4

In [ ]:
ta = np.arange(nbPoints)
tb = np.arange(nbPoints*upSampling)/upSampling

plt.figure()

plt.plot(tb, np.real(b), 'o', label="b real")
plt.plot(tb, np.imag(b), 'o', label="b imag")
plt.plot(tb, np.abs(b), 'o',label="b abs")

plt.plot(ta, np.real(a), '.-', label="a real")
plt.plot(ta, np.imag(a), '.-', label="a imag")
plt.plot(ta, np.abs(a), '.-', label="a abs")

plt.legend()

In [ ]:
a = np.zeros(4)

In [ ]:
a[0] = 0
a[1] = 1
a[2] = 0
a[3] = 0

In [ ]:
a

In [ ]:
ffta = np.fft.fft(a)

In [ ]:
ffta

In [ ]:
plt.figure()
plt.plot(np.angle(ffta))